In [12]:
import os
import json
import librosa
import numpy as np
import pandas as pd
from pathlib import Path

# 📁 JSON 및 WAV 파일 경로 설정
json_folder_car = Path("C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/stereo_folder/1.자동차")
json_folder_bike = Path("C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/stereo_folder/2.이륜자동차")

wav_folder_car = Path("C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/stereo_folder/1.자동차_stereo")
wav_folder_bike = Path("C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/stereo_folder/2.이륜자동차_stereo")

# 📌 SPL 기준값 설정
SPL_REFERENCE = 20e-6  # 0dB 기준

# 📌 JSON 데이터 불러오기
def load_json_data(json_path):
    try:
        with open(json_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        return data
    except Exception as e:
        print(f"⚠️ JSON 로드 실패: {json_path}, 오류: {e}")
        return None

# 📌 WAV 파일 분석 (데시벨, 방향)
def analyze_wav(wav_path):
    try:
        y, sr = librosa.load(wav_path, sr=None, mono=False)
        if len(y.shape) == 1:  # 모노일 경우
            is_stereo = False
            left_channel = right_channel = y
        else:  # 스테레오일 경우
            is_stereo = True
            left_channel, right_channel = y
        
        # 🔹 RMS SPL 계산 (수정된 공식 적용)
        rms_total = np.sqrt(np.mean(y ** 2))
        spl_total = 20 * np.log10((rms_total / SPL_REFERENCE) + 1e-6)  # ✔ 개선된 공식 적용

        # 🔹 방향 판별 (스테레오만 적용)
        if is_stereo:
            rms_left = np.sqrt(np.mean(left_channel ** 2))
            rms_right = np.sqrt(np.mean(right_channel ** 2))
            spl_left = 20 * np.log10((rms_left / SPL_REFERENCE) + 1e-6)
            spl_right = 20 * np.log10((rms_right / SPL_REFERENCE) + 1e-6)

            if abs(spl_left - spl_right) < 2:
                direction = "중앙"
            else:
                direction = "좌" if spl_left > spl_right else "우"
        else:
            direction = None  # 모노일 경우 방향 판별 불가

        return round(spl_total, 1), direction
    except Exception as e:
        print(f"⚠️ WAV 분석 실패: {wav_path}, 오류: {e}")
        return None, None

# 📌 WAV와 JSON 매칭 및 비교 실행
comparison_results = []

for wav_folder, json_folder in [(wav_folder_car, json_folder_car), (wav_folder_bike, json_folder_bike)]:
    for wav_file in wav_folder.glob("*.wav"):
        base_name = wav_file.stem.rsplit("_", 1)[0]  # `_1` 제거
        json_file = json_folder / f"{base_name}.json"

        if not json_file.exists():
            print(f"⚠️ JSON 없음: {json_file}")
            continue

        # JSON 데이터 로드
        json_data = load_json_data(json_file)
        if not json_data:
            continue
        
        # JSON 정보 추출
        json_decibel = json_data["annotations"][0]["decibel"]
        json_direction = json_data["environment"]["direction"]

        # WAV 분석 실행
        wav_predicted_decibel, wav_predicted_direction = analyze_wav(wav_file)

        # 비교 결과 저장
        comparison_results.append({
            "wav_file": wav_file.name,
            "json_decibel": json_decibel,
            "wav_predicted_decibel": round(wav_predicted_decibel) if wav_predicted_decibel else "None",
            "json_direction": json_direction,
            "wav_predicted_direction": wav_predicted_direction if wav_predicted_direction else "None",
        })

# 📊 결과 CSV 저장
output_csv_path = "C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/comparison_results_decibel_direction_fixed.csv"
comparison_df = pd.DataFrame(comparison_results)
comparison_df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")
print(f"✅ 결과 CSV 저장 완료: {output_csv_path}")



✅ 결과 CSV 저장 완료: C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/comparison_results_decibel_direction_fixed.csv


In [14]:
import os
import json
import librosa
import numpy as np
import pandas as pd
from scipy.signal import butter, lfilter
from pathlib import Path

# 📁 JSON 및 WAV 파일 경로 설정
json_folder_car = Path("C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/stereo_folder/1.자동차")
json_folder_bike = Path("C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/stereo_folder/2.이륜자동차")

wav_folder_car = Path("C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/stereo_folder/1.자동차_stereo")
wav_folder_bike = Path("C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/stereo_folder/2.이륜자동차_stereo")

# 📌 SPL 기준값 설정
SPL_REFERENCE = 20e-6  # 0dB 기준

# 📌 고주파 필터 (2kHz 이상)
def highpass_filter(y, sr, cutoff=2000, order=4):
    nyquist = 0.5 * sr
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return lfilter(b, a, y)

# 📌 JSON 데이터 불러오기
def load_json_data(json_path):
    try:
        with open(json_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        return data
    except Exception as e:
        print(f"⚠️ JSON 로드 실패: {json_path}, 오류: {e}")
        return None

# 📌 WAV 파일 분석 (데시벨, 방향)
def analyze_wav(wav_path, apply_highpass=False):
    try:
        y, sr = librosa.load(wav_path, sr=None, mono=False)
        if len(y.shape) == 1:  # 모노일 경우
            is_stereo = False
            left_channel = right_channel = y
        else:  # 스테레오일 경우
            is_stereo = True
            left_channel, right_channel = y
        
        # 🔹 RMS SPL 계산 (전체 대역)
        rms_total = np.sqrt(np.mean(y ** 2))
        spl_total = 20 * np.log10((rms_total / SPL_REFERENCE) + 1e-6)  # ✔ 개선된 공식 적용

        # 🔹 방향 판별 (스테레오만 적용)
        if is_stereo:
            if apply_highpass:  # 🔥 고주파 필터 적용
                left_channel = highpass_filter(left_channel, sr)
                right_channel = highpass_filter(right_channel, sr)

            rms_left = np.sqrt(np.mean(left_channel ** 2))
            rms_right = np.sqrt(np.mean(right_channel ** 2))
            spl_left = 20 * np.log10((rms_left / SPL_REFERENCE) + 1e-6)
            spl_right = 20 * np.log10((rms_right / SPL_REFERENCE) + 1e-6)

            if abs(spl_left - spl_right) < 1:  # 🔥 기준값을 1dB로 조정 (더 정밀하게)
                direction = "중앙"
            else:
                direction = "왼쪽" if spl_left > spl_right else "오른쪽"
        else:
            direction = None  # 모노일 경우 방향 판별 불가

        return round(spl_total, 1), direction
    except Exception as e:
        print(f"⚠️ WAV 분석 실패: {wav_path}, 오류: {e}")
        return None, None

# 📌 WAV와 JSON 매칭 및 비교 실행
comparison_results = []

for wav_folder, json_folder in [(wav_folder_car, json_folder_car), (wav_folder_bike, json_folder_bike)]:
    for wav_file in wav_folder.glob("*.wav"):
        base_name = wav_file.stem.rsplit("_", 1)[0]  # `_1` 제거
        json_file = json_folder / f"{base_name}.json"

        if not json_file.exists():
            print(f"⚠️ JSON 없음: {json_file}")
            continue

        # JSON 데이터 로드
        json_data = load_json_data(json_file)
        if not json_data:
            continue
        
        # JSON 정보 추출
        json_decibel = json_data["annotations"][0]["decibel"]
        json_direction = json_data["environment"]["direction"]

        # WAV 분석 실행 (고주파 필터 적용)
        wav_predicted_decibel, wav_predicted_direction = analyze_wav(wav_file, apply_highpass=True)

        # 비교 결과 저장
        comparison_results.append({
            "wav_file": wav_file.name,
            "json_decibel": json_decibel,
            "wav_predicted_decibel": round(wav_predicted_decibel) if wav_predicted_decibel else "None",
            "json_direction": json_direction,
            "wav_predicted_direction": wav_predicted_direction if wav_predicted_direction else "None",
        })

# 📊 결과 CSV 저장
output_csv_path = "C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/comparison_results_highpass_direction.csv"
comparison_df = pd.DataFrame(comparison_results)
comparison_df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")
print(f"✅ 결과 CSV 저장 완료: {output_csv_path}")



✅ 결과 CSV 저장 완료: C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/comparison_results_highpass_direction.csv


In [21]:
import pandas as pd

# 📁 파일 경로 수정 (raw string 사용)
file_path = r"C:\Users\yi260\OneDrive\바탕 화면\WAV & JSON 비교 폴더\comparison_results_highpass_direction.csv"

# CSV 파일 로드
df = pd.read_csv(file_path)

# 1. 전체 JSON 데시벨과 예측한 데시벨의 차이 평균값 계산
df["decibel_diff"] = abs(df["json_decibel"] - df["wav_predicted_decibel"])
decibel_diff_mean = df["decibel_diff"].mean()

# 2. JSON 데이터의 방향 비율 계산
json_direction_counts = df["json_direction"].value_counts(normalize=True) * 100

# 3. 예측된 방향의 비율 계산
wav_direction_counts = df["wav_predicted_direction"].value_counts(normalize=True) * 100

# 결과 출력
decibel_diff_mean, json_direction_counts, wav_direction_counts






(19.51058370750481,
 json_direction
 우    75.240539
 좌    24.695318
 위     0.064144
 Name: proportion, dtype: float64,
 wav_predicted_direction
 중앙     47.851187
 오른쪽    29.377806
 왼쪽     22.771007
 Name: proportion, dtype: float64)

In [23]:
import pandas as pd

# CSV 파일 로드
file_path = r"C:\Users\yi260\OneDrive\바탕 화면\WAV & JSON 비교 폴더\comparison_results_highpass_direction.csv"
df = pd.read_csv(file_path)

# 1. 전체 JSON 데시벨과 예측한 데시벨의 차이 평균값 계산
df["decibel_diff"] = abs(df["json_decibel"] - df["wav_predicted_decibel"])

# 2. 오차범위 10% 이내에 맞는 경우 찾기
df["decibel_within_10_percent"] = df["decibel_diff"] <= (df["json_decibel"] * 0.1)

# 3. 10% 이내로 맞는 비율 계산
percentage_within_10_percent = df["decibel_within_10_percent"].mean() * 100

# 결과 출력
percentage_within_10_percent



35.9204618345093

In [6]:
import os
import json
import librosa
import numpy as np
import pandas as pd
from pathlib import Path

# 🔹 SPL 기준값 (소음 유형별 데시벨 기준)
DB_REFERENCE = {
    "차량 경적": 100,
    "이륜차 경적": 100,
    "사이렌": 100,
    "차량 주행음": 90,
    "이륜차 주행음": 90,
    "기타 소음": 85,
}

SPL_REFERENCE = 20e-6  # 0dB 기준 음압

# 📁 폴더 경로 설정
json_folder_car = Path("C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/stereo_folder/1.자동차")
json_folder_bike = Path("C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/stereo_folder/2.이륜자동차")
wav_folder_car = Path("C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/stereo_folder/1.자동차_stereo")
wav_folder_bike = Path("C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/stereo_folder/2.이륜자동차_stereo")

# 📌 JSON 파일 매칭을 위한 딕셔너리 생성
json_data_dict = {}

def load_json_data(json_folder):
    """ JSON 데이터 로드 """
    for json_file in json_folder.glob("*.json"):
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)
            file_name = data["audio"]["fileName"].replace(".wav", "")  # JSON의 파일명 추출
            json_data_dict[file_name] = {
                "decibel": float(data["annotations"][0]["decibel"]),
                "distance": data["environment"]["distance"],
                "direction": data["environment"]["direction"]
            }

# 🔹 JSON 데이터 로드 (자동차 + 이륜차)
load_json_data(json_folder_car)
load_json_data(json_folder_bike)

# 📌 분석 결과 저장 리스트
comparison_results = []

def analyze_wav(file_path, json_data):
    """ WAV 파일 분석 및 JSON과 비교 """
    try:
        y, sr = librosa.load(file_path, sr=None, mono=False, dtype=np.float32)  # `norm` 제거 및 `dtype` 변경

        # 🔹 스테레오 여부 확인
        is_stereo = len(y.shape) == 2 and y.shape[0] == 2

        # 🔹 데시벨 계산 (RMS SPL & Peak SPL)
        rms_total = np.sqrt(np.mean(y ** 2))
        peak_amplitude = np.max(np.abs(y))
        rms_spl = 20 * np.log10(rms_total / SPL_REFERENCE + 1e-6)
        peak_spl = 20 * np.log10(peak_amplitude / SPL_REFERENCE + 1e-6)

        # 🔹 거리 계산 (감쇠율 적용)
        category = json_data["direction"]
        db_ref = DB_REFERENCE.get(category, 85)
        estimated_distance = 1 * (10 ** ((db_ref - rms_spl) / 20))
        estimated_distance = max(0.1, min(estimated_distance, 50))  # 50m 이상은 '50m 이상'으로 표기

        # 🔹 방향 판별 (좌/우 SPL 차이 계산)
        if is_stereo:
            left_spl = 20 * np.log10(np.sqrt(np.mean(y[0] ** 2)) / SPL_REFERENCE + 1e-6)
            right_spl = 20 * np.log10(np.sqrt(np.mean(y[1] ** 2)) / SPL_REFERENCE + 1e-6)
            direction = "왼쪽" if left_spl > right_spl else "오른쪽"
        else:
            direction = "알 수 없음"

        # 🔹 JSON과 비교
        json_distance = json_data["distance"]
        json_decibel = json_data["decibel"]
        json_direction = json_data["direction"]

        match_distance = abs(float(json_distance.replace("m", "")) - estimated_distance) < 5  # 5m 이내 차이면 매칭
        match_decibel = abs(json_decibel - rms_spl) < 5  # 5dB 이내 차이면 매칭
        match_direction = (json_direction == direction) if is_stereo else "N/A"

        comparison_results.append({
            "wav_file": file_path.name,
            "json_distance": json_distance,
            "wav_predicted_distance": f"{round(estimated_distance, 1)}m",
            "json_decibel": json_decibel,
            "wav_predicted_decibel": round(rms_spl, 1),
            "json_direction": json_direction,
            "wav_predicted_direction": direction,
            "match_distance": match_distance,
            "match_decibel": match_decibel,
            "match_direction": match_direction
        })
    
    except Exception as e:
        print(f"⚠️ WAV 분석 실패: {file_path}, 오류: {e}")

# 🔹 WAV 파일 분석 (자동차 + 이륜차)
for wav_file in wav_folder_car.glob("*.wav"):
    base_name = wav_file.stem.rsplit("_", 1)[0]  # `_1` 제거
    if base_name in json_data_dict:
        analyze_wav(wav_file, json_data_dict[base_name])

for wav_file in wav_folder_bike.glob("*.wav"):
    base_name = wav_file.stem.rsplit("_", 1)[0]
    if base_name in json_data_dict:
        analyze_wav(wav_file, json_data_dict[base_name])

# 📊 결과 데이터프레임 생성
comparison_df = pd.DataFrame(comparison_results)

# 📁 CSV 파일로 저장 (파일 확인 가능)
output_csv_path = "C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/comparison_results_fixed.csv"
comparison_df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")

# 📌 데이터프레임 출력
from IPython.display import display
display(comparison_df)

print(f"✅ 결과 CSV 저장 완료: {output_csv_path}")






,wav_file,json_distance,wav_predicted_distance,json_decibel,wav_predicted_decibel,json_direction,wav_predicted_direction,match_distance,match_decibel,match_direction
0,1.자동차_1000_1.wav,30m,3.9m,122.0,73.2,우,왼쪽,False,False,False
1,1.자동차_1001_1.wav,10m,3.2m,105.0,74.8,좌,오른쪽,False,False,False
2,1.자동차_1008_1.wav,13m,10.2m,71.0,64.8,우,오른쪽,True,False,False
3,1.자동차_1010_1.wav,10m,6.4m,74.0,68.8,좌,오른쪽,True,False,False
4,1.자동차_1011_1.wav,24m,2.9m,120.0,75.7,우,오른쪽,False,False,False
...,...,...,...,...,...,...,...,...,...,...
1554,2.이륜자동차_3078_1.wav,11m,19.5m,71.0,59.2,우,오른쪽,False,False,False
1555,2.이륜자동차_3081_1.wav,5m,4.2m,74.0,72.5,우,오른쪽,True,True,False
1556,2.이륜자동차_3082_1.wav,1m,4.9m,118.0,71.1,우,왼쪽,True,False,False
1557,2.이륜자동차_3083_1.wav,10m,1.0m,113.0,85.1,좌,왼쪽,False,False,False


✅ 결과 CSV 저장 완료: C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/comparison_results_fixed.csv


In [7]:
import os
import json
import librosa
import numpy as np
import pandas as pd
from pathlib import Path

# 🔹 SPL 기준값
SPL_REFERENCE = 20e-6  # 0dB 기준 음압

# 📁 폴더 경로 설정
json_folder_car = Path("C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/stereo_folder/1.자동차")
json_folder_bike = Path("C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/stereo_folder/2.이륜자동차")
wav_folder_car = Path("C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/stereo_folder/1.자동차_stereo")
wav_folder_bike = Path("C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/stereo_folder/2.이륜자동차_stereo")

# 📌 JSON 파일 매칭을 위한 딕셔너리 생성
json_data_dict = {}

def load_json_data(json_folder):
    """ JSON 데이터 로드 """
    for json_file in json_folder.glob("*.json"):
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)
            file_name = data["audio"]["fileName"].replace(".wav", "")  # JSON의 파일명 추출
            json_data_dict[file_name] = {
                "decibel": float(data["annotations"][0]["decibel"]),
                "direction": data["environment"]["direction"]
            }

# 🔹 JSON 데이터 로드 (자동차 + 이륜차)
load_json_data(json_folder_car)
load_json_data(json_folder_bike)

# 📌 분석 결과 저장 리스트
comparison_results = []

def analyze_wav(file_path, json_data):
    """ WAV 파일 분석 및 JSON과 비교 """
    try:
        y, sr = librosa.load(file_path, sr=None, mono=False, dtype=np.float32)  # `norm` 제거 및 `dtype` 변경

        # 🔹 스테레오 여부 확인
        is_stereo = len(y.shape) == 2 and y.shape[0] == 2

        # 🔹 데시벨 계산 (최고 SPL & 평균 SPL)
        peak_amplitude = np.max(np.abs(y))
        peak_spl = 20 * np.log10((peak_amplitude / SPL_REFERENCE) + 1e-6)

        rms_total = np.sqrt(np.mean(y ** 2))
        rms_spl = 20 * np.log10((rms_total / SPL_REFERENCE) + 1e-6)

        # 🔹 방향 판별 (좌/우 SPL 차이 계산)
        if is_stereo:
            left_spl = 20 * np.log10(np.sqrt(np.mean(y[0] ** 2)) / SPL_REFERENCE + 1e-6)
            right_spl = 20 * np.log10(np.sqrt(np.mean(y[1] ** 2)) / SPL_REFERENCE + 1e-6)

            # 📌 JSON 방향 형식(`좌/우/위`)에 맞춰 변환
            if abs(left_spl - right_spl) < 2:
                predicted_direction = "중앙"
            else:
                predicted_direction = "좌" if left_spl > right_spl else "우"
        else:
            predicted_direction = "중앙"  # 모노 오디오는 방향 정보 없음

        # 🔹 JSON과 비교
        json_decibel = json_data["decibel"]
        json_direction = json_data["direction"]

        comparison_results.append({
            "wav_file": file_path.name,
            "json_decibel": json_decibel,
            "wav_peak_spl": round(peak_spl, 1),
            "wav_rms_spl": round(rms_spl, 1),
            "json_direction": json_direction,
            "wav_predicted_direction": predicted_direction,
            "spl_difference_peak": round(abs(peak_spl - json_decibel), 1),
            "spl_difference_rms": round(abs(rms_spl - json_decibel), 1),
            "match_direction": json_direction == predicted_direction
        })
    
    except Exception as e:
        print(f"⚠️ WAV 분석 실패: {file_path}, 오류: {e}")

# 🔹 WAV 파일 분석 (자동차 + 이륜차)
for wav_file in wav_folder_car.glob("*.wav"):
    base_name = wav_file.stem.rsplit("_", 1)[0]  # `_1` 제거
    if base_name in json_data_dict:
        analyze_wav(wav_file, json_data_dict[base_name])

for wav_file in wav_folder_bike.glob("*.wav"):
    base_name = wav_file.stem.rsplit("_", 1)[0]
    if base_name in json_data_dict:
        analyze_wav(wav_file, json_data_dict[base_name])

# 📊 결과 데이터프레임 생성
comparison_df = pd.DataFrame(comparison_results)

# 📁 CSV 파일로 저장 (파일 확인 가능)
output_csv_path = "C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/comparison_results_decibel_only.csv"
comparison_df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")

# 📌 데이터프레임 출력
from IPython.display import display
display(comparison_df)

print(f"✅ 결과 CSV 저장 완료: {output_csv_path}")


,wav_file,json_decibel,wav_peak_spl,wav_rms_spl,json_direction,wav_predicted_direction,spl_difference_peak,spl_difference_rms,match_direction
0,1.자동차_1000_1.wav,122.0,94.0,73.2,우,좌,28.0,48.8,False
1,1.자동차_1001_1.wav,105.0,90.9,74.8,좌,중앙,14.1,30.2,False
2,1.자동차_1008_1.wav,71.0,82.2,64.8,우,중앙,11.2,6.2,False
3,1.자동차_1010_1.wav,74.0,85.0,68.8,좌,중앙,11.0,5.2,False
4,1.자동차_1011_1.wav,120.0,90.8,75.7,우,중앙,29.2,44.3,False
...,...,...,...,...,...,...,...,...,...
1554,2.이륜자동차_3078_1.wav,71.0,83.6,59.2,우,중앙,12.6,11.8,False
1555,2.이륜자동차_3081_1.wav,74.0,94.0,72.5,우,우,20.0,1.5,True
1556,2.이륜자동차_3082_1.wav,118.0,90.2,71.1,우,중앙,27.8,46.9,False
1557,2.이륜자동차_3083_1.wav,113.0,94.0,85.1,좌,중앙,19.0,27.9,False


✅ 결과 CSV 저장 완료: C:/Users/yi260/OneDrive/바탕 화면/WAV & JSON 비교 폴더/comparison_results_decibel_only.csv
